# <a name="hello-world"></a> A hello-world example of dflow

Define a Python OP

In [22]:
from pathlib import Path
from dflow.python import OP, Artifact
import numpy as np
import os
import pytz, dateutil
import torch
@OP.function
def duplicate(num: int, foo: Artifact(Path)) -> {"num": int, "bar": Artifact(Path)}:
    with open(foo, "r") as f:
        content = f.read()
        print(content)
    with open("bar.txt", "w") as f:
        f.write(content * 2)
    num_ = num * 2
    print("num_", num_)
    return {
        "num": num_,
        "bar": Path("bar.txt"),
    }

Upload the input file

In [23]:
from dflow import upload_artifact
import deepchem.data

art = upload_artifact("foo.txt")

In [24]:
from dflow import LocalArtifact

# Assuming 'trainBBBP.csv' is in the current working directory
foo_art = LocalArtifact(local_path="foo.txt")
print(foo_art)

In [25]:
import os
print(os.getcwd())

/Users/star/Desktop/DeepPotential/Dflow_Star


Instantiate a step with the dispatcher executor and the uploaded artifact

In [26]:
from dflow.python import PythonOPTemplate
from dflow import Step
import sys

step1 = Step(
    name="step1",
    template=PythonOPTemplate(
        duplicate, 
        image="python:3.11",
        ),
    parameters={"num": 1},
    artifacts={"foo": art},
)

step2 = Step(
    name="step2",
    template=PythonOPTemplate(
        duplicate, 
        f"python:{sys.version_info.major}.{sys.version_info.minor}",
        ),
    parameters={"num": step1.outputs.parameters["num"]},
    artifacts={"foo": art},
)

Add step into a workflow

In [27]:
from dflow import Workflow

wf = Workflow(name="python-test")
wf.add(step1)
wf.add(step2)

Submit the workflow

In [28]:
wf.submit()

Workflow has been submitted (ID: python-test-d6vnt, UID: 417072cb-b31b-477c-a728-d8dd2c4736b7)
Workflow link: https://127.0.0.1:2746/workflows/argo/python-test-d6vnt


{'metadata': {'name': 'python-test-d6vnt', 'generateName': 'python-test-', 'namespace': 'argo', 'uid': '417072cb-b31b-477c-a728-d8dd2c4736b7', 'resourceVersion': '232239', 'generation': 1, 'creationTimestamp': '2024-01-14T07:15:25Z', 'labels': {'workflows.argoproj.io/creator': 'system-serviceaccount-argo-argo-server'}, 'managedFields': [{'manager': 'argo', 'operation': 'Update', 'apiVersion': 'argoproj.io/v1alpha1', 'time': '2024-01-14T07:15:25Z', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:workflows.argoproj.io/creator': {}}}, 'f:spec': {}, 'f:status': {}}}]}, 'spec': {'templates': [{'name': 'duplicate-0f0iv', 'inputs': {'parameters': [{'name': 'num', 'description': '{"type": "int"}'}], 'artifacts': [{'name': 'foo', 'path': '/tmp/inputs/artifacts/foo'}, {'name': 'dflow_python_packages', 'path': '/tmp/inputs/artifacts/dflow_python_packages'}]}, 'outputs': {'parameters': [{'name': 'num', 'valueFrom': {'path': '/tmp/outputs/paramet

# <a name="Argo-UI"></a> Argo UI

After finishing the previous steps, we can access the workflow we just ran on the UI (https://127.0.0.1:2746)